In [1]:
import mo_descriptor as md
import nn_frame as nn
import numpy as np
import subprocess
subprocess.run('export TF_INTRA_OP_PARALLELISM_THREADS=12', shell=True)

2022-10-28 21:57:20.597786: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-28 21:57:20.597915: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


CompletedProcess(args='export TF_INTRA_OP_PARALLELISM_THREADS=12', returncode=0)

In [2]:
'''
prepare data_set
1. make mo_pair descriptor
'''
# x_shift = np.arange(0, 4.1, 0.1)
# y_shift = np.arange(0, 4.1, 0.1)
# z_shift = np.zeros(x_shift.shape)
# # the original mo, e.g. homo
# homo = md.MO_descriptor('data/homo-s0.cube').make()
# lumo = md.MO_descriptor('data/lumo-s0.cube').make()

# # for the original pair of one mo and itself
# homo_pair = md.MO_pair_descriptor(homo, homo).make()
# lumo_pair = md.MO_pair_descriptor(lumo, lumo).make()

# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)

# homo_ = np.zeros(homo.shape)
# lumo_ = np.zeros(lumo.shape)

# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_[:,0] = np.add(homo[:,0],0)
#         homo_[:,1] = np.add(homo[:,1],i)
#         homo_[:,2] = np.add(homo[:,2],j)
#         homo_[:,3] = np.add(homo[:,3],3.5)
        
#         homo_pair_ = md.MO_pair_descriptor(homo, homo_).make()
#         homo_pairs[idx] = homo_pair_
        
#         lumo_[:,0] = np.add(lumo[:,0],0)
#         lumo_[:,1] = np.add(lumo[:,1],i)
#         lumo_[:,2] = np.add(lumo[:,2],j)
#         lumo_[:,3] = np.add(lumo[:,3],3.5)
        
#         lumo_pair_ = md.MO_pair_descriptor(lumo, lumo_).make()
#         lumo_pairs[idx] = lumo_pair_
        
# def dir_mat(mat):
#     mat_shape = mat.shape
#     mat_ = mat.flatten()
#     for ii, i in enumerate(mat_):
#         if i > 1e-6:
#             mat_[ii] = 1
#         elif (i < 1e-6) and (i > -1e-6):
#             mat_[ii] = -1
#         elif i < -1e-6:
#             mat_[ii] = -1
#     return mat_.reshape(mat_shape)

# direct = dir_mat(homo_pair)

# # for the shifted pair
# homo_pairs = np.zeros((len(x_shift)*len(y_shift),) + homo_pair.shape)
# lumo_pairs = np.zeros((len(x_shift)*len(y_shift),) + lumo_pair.shape)
# for ii, i in enumerate(x_shift):
#     for jj, j in enumerate(y_shift):
#         idx = ii * len(y_shift) + jj
#         homo_pairs[idx][0] = homo_pair[0]
#         homo_pairs[idx][1] = np.add(homo_pair[1],i*direct[1])
#         homo_pairs[idx][2] = np.add(homo_pair[2],j*direct[2])
#         homo_pairs[idx][3] = homo_pair[3]
#         lumo_pairs[idx][0] = lumo_pair[0]
#         lumo_pairs[idx][1] = np.add(lumo_pair[1],i)
#         lumo_pairs[idx][2] = np.add(lumo_pair[2],j)
#         lumo_pairs[idx][3] = lumo_pair[3]
# np.save('homo_homo_pair.npy', homo_pairs)
# np.save('lumo_lumo_pair.npy', lumo_pairs)
homo_pairs = np.load('homo_homo_pair.npy')
lumo_pairs = np.load('lumo_lumo_pair.npy')

In [3]:
'''
2. read coupling
'''
raw_data = np.loadtxt('../ML_Coupling/results.csv', delimiter=',',comments='#')
# c_homo = np.add(raw_data[:,2], raw_data[:,3]) * 1/2
# c_lumo = np.add(raw_data[:,4], raw_data[:,5]) * 1/2
c_homo = abs(raw_data[:,3])
c_lumo = abs(raw_data[:,4])

In [4]:
# x1 = homo_pairs[:,0,:,:]
# x2 = homo_pairs[:,1,:,:]
# x3 = homo_pairs[:,2,:,:]
# x4 = homo_pairs[:,3,:,:]
# x = np.einsum('aij,aij,aij,aij->aij', x1, x2, x3, x4)

train_homo_pairs = homo_pairs
train_lumo_pairs = lumo_pairs

train_c_homo = -np.log(c_homo)
train_c_lumo = -np.log(c_lumo)

test_homo_pairs = homo_pairs[1200:]
test_lumo_pairs = lumo_pairs[1200:]

test_c_homo = c_homo[1200:].reshape((len(c_homo[1200:]),1))
test_c_lumo = c_lumo[1200:]

In [5]:
import copy
train_homo = copy.deepcopy(train_homo_pairs)
train_chomo = copy.deepcopy(train_c_homo)
print(len(train_chomo))
index = np.random.choice(1681,size=481, replace=False)
train_homo_ = np.delete(train_homo,index,0)
train_chomo_ = np.delete(train_chomo,index,0)
print(len(train_chomo_))

1681
1200


In [ ]:
setting = {'activation':'tanh', 'nn_shape':(256,256,256), 'batch_size':len(train_chomo_), 'training_steps':200000,\
'learning_rate': 0.00001, 'decay_rate':0.95, 'decay_per_steps':1000, 'save_step':100, 'drop_rate':0, 'save_path':'./save',\
'seed':None, 'debug_traj':False}
NN = nn.NN(setting_dict=setting)
NN.train(train_homo_,train_chomo_)

2022-10-28 21:57:22.311661: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-10-28 21:57:22.311785: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Deng-PC): /proc/driver/nvidia/version does not exist
2022-10-28 21:57:22.312357: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


loss:  41.7953339
training step:     0
loss:  36.6198235
training step:   100
loss:  36.2871704
training step:   200
loss:  36.0032349
training step:   300
loss:  35.7298813
training step:   400
loss:  35.4590683
training step:   500
loss:  35.1882591
training step:   600
loss:  34.9155846
training step:   700
loss:  34.6392
training step:   800
loss:  34.3566742
training step:   900
loss:  34.06847
training step:  1000
loss:  33.7735786
training step:  1100
loss:  33.4712257
training step:  1200
loss:  33.1608925
training step:  1300
loss:  32.8424492
training step:  1400
loss:  32.5160561
training step:  1500
loss:  32.1817207
training step:  1600
loss:  31.8395252
training step:  1700
loss:  31.489851
training step:  1800
loss:  31.1330051
training step:  1900
loss:  30.7692375
training step:  2000
loss:  30.3990421
training step:  2100
loss:  30.0228901
training step:  2200
loss:  29.6411858
training step:  2300
loss:  29.2543106
training step:  2400
loss:  28.862709
training step:

In [ ]:
iall = np.arange(1681)
idiff = np.setdiff1d(iall,index)
test_homo = np.delete(copy.deepcopy(train_homo_pairs),idiff,0)
test_chomo = np.delete(copy.deepcopy(train_chomo),idiff,0)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
%matplotlib widget

error = np.mean(np.multiply(NN.model(train_homo_pairs, training=False).numpy().reshape((len(train_homo_pairs),))-train_c_homo, np.power(train_c_homo,-1))*100)
x = np.linspace(0, 4, 41)
y = np.linspace(0, 4, 41)
Z = NN.model(train_homo_pairs, training=False).numpy().reshape((41,41)).T
Z1 = c_homo.reshape((41,41)).T

fix, ax = plt.subplots()
ax.contourf(x,y, np.exp(-Z))
ax.set_title('Error: %5.3f%%'%error)
ax.set_aspect('equal')
# plt.savefig('homo_pred.png')

In [ ]:
fix, ax = plt.subplots()
ax.contourf(x,y, Z1)
ax.set_title('True')
ax.set_aspect('equal')
# plt.savefig('homo_true.png')

In [ ]:
C = []
for i in x:
    for j in y:
        C_ = (i,j)
        C.append(C_)
C = np.array(C)
C_ = copy.deepcopy(C)
C = np.delete(C,index,0)
C__ = np.delete(C_,idiff,0)
print(len(C__))

Z = NN.model(test_homo, training=False).numpy()

fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
ax.scatter(C__[:, 0], C__[:, 1], c=np.exp(-Z),s=62, marker=',')

ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')

error = np.mean(np.multiply(NN.model(test_homo, training=False).numpy().reshape((len(test_homo),))-test_chomo, np.power(test_chomo,-1))*100)
plt.title('ML with error of test set: %5.3f %%'%error)
plt.tight_layout()
plt.show()

# plt.savefig('test.jpg')

In [ ]:
Z1 = train_chomo
fig = plt.figure(figsize=(6,6))
ax = fig.add_subplot()
ax.scatter(C_[:, 0], C_[:, 1], c=np.exp(-Z1),s=62, marker=',')

ax.set_xlabel('X Axis')
ax.set_ylabel('Y Axis')

# plt.title('HOMO of Naphthalene')
plt.tight_layout()
plt.show()
# plt.savefig('test.jpg')

In [ ]:
NN.model.summary()

In [ ]:
error1 = np.mean(np.multiply(NN.model(train_homo_pairs, training=False).numpy().reshape((len(train_homo_pairs),))-train_c_homo, np.power(train_c_homo,-1))*100)
error2 = np.mean(np.multiply(NN.model(train_homo_, training=False).numpy().reshape((len(train_homo_),))-train_chomo_, np.power(train_chomo_,-1))*100)
error3 = np.mean(np.multiply(NN.model(test_homo, training=False).numpy().reshape((len(test_homo),))-test_chomo, np.power(test_chomo,-1))*100)
print('Error of full data set: %5.3f %% \nError of training set with %d samples: %5.3f %% \nError of testing set with %d samples: %5.3f %% '%(error1,len(train_homo_),error2,len(test_homo),error3))

In [ ]:
import tensorflow as tf
X = np.linspace(0,9,10)
Y = np.linspace(0,9,10)
bs = 3
data_set = tf.data.Dataset.from_tensor_slices((X,Y)).shuffle(bs,reshuffle_each_iteration=None).batch(bs)
for x,y in data_set:
    print(x.numpy())